In [1]:
import pandas as pd
import os
from datetime import datetime
import sqlite3
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.inspection import plot_partial_dependence
from sklearn.inspection import partial_dependence 


%matplotlib inline

In [2]:
# police beats interactive map - put in address and it tells you the beat!

# https://www.seattle.gov/police/community-policing/mcpp-location-map
    
    

In [2]:
# function to convert datetime object to 4 - for force ONLY
def datetime(df, col_as_str):
    yearmonthday = df[col_as_str].astype(str).str.split(" ")
    ymd = yearmonthday.apply(lambda x: x[0]).str.split("/")
    year = ymd.apply(lambda x: x[2])
    month = ymd.apply(lambda x: x[0])
    day = ymd.apply(lambda x: x[1])
    time = ymd = yearmonthday.apply(lambda x: x[1]+x[2]).astype(str)
    df["year"] = year
    df["month"] = month
    df["day"] = day
    tim = pd.to_datetime(time)
    df["hour"] = tim.apply(lambda x: pd.datetime.strftime(x, '%H'))
    return df



In [3]:
force = pd.read_csv("data/Use_Of_Force.csv")
len(force) -  len(force[force["Beat"] == '-']), len(force)

(9422, 9838)

In [4]:
# change to epoch time
force = datetime(force,"Occured_date_time" )
foo = pd.to_datetime( force.Occured_date_time )
epoch_second = foo.map(lambda x:x.value/1e9)

force["epoch"] = epoch_second


In [6]:
# EDA - use to verify percentages later

'''lf3 = ["Level 3 - Use of Force", 'Level 3 - OIS']
lf1 = ["Level 2 - Use of Force", "Level 1 - Use of Force"]
f1 = force[force["Incident_Type"].isin(lf1)]
f3 = force[force["Incident_Type"].isin(lf3)]

top10 = f3.Officer_ID.value_counts()[:10].index
len(force[force["Officer_ID"].isin(top10)])
for officer in top10:
    off = f3[f3["Officer_ID"] == officer]
    beats = off.Beat.value_counts()
    print(officer, beats)  '''

# could look at incidents and see if they are related to officer OR beat or some of each

'lf3 = ["Level 3 - Use of Force", \'Level 3 - OIS\']\nlf1 = ["Level 2 - Use of Force", "Level 1 - Use of Force"]\nf1 = force[force["Incident_Type"].isin(lf1)]\nf3 = force[force["Incident_Type"].isin(lf3)]\n\ntop10 = f3.Officer_ID.value_counts()[:10].index\nlen(force[force["Officer_ID"].isin(top10)])\nfor officer in top10:\n    off = f3[f3["Officer_ID"] == officer]\n    beats = off.Beat.value_counts()\n    print(officer, beats)  '

In [7]:
# EDA - use to verify percentages later

'''beats = force.groupby("Beat")
vc = beats["Incident_Type"].value_counts()
vc = pd.DataFrame(vc)
vc.values
OIS_dict = {}
#vc.index[3][1]
for i in range(vc.shape[0]):
    if vc.index[i][1]== "Level 3 - OIS":
        OIS_dict[vc.index[i][0]] = vc.values[i][0]
OIS_dict'''

'beats = force.groupby("Beat")\nvc = beats["Incident_Type"].value_counts()\nvc = pd.DataFrame(vc)\nvc.values\nOIS_dict = {}\n#vc.index[3][1]\nfor i in range(vc.shape[0]):\n    if vc.index[i][1]== "Level 3 - OIS":\n        OIS_dict[vc.index[i][0]] = vc.values[i][0]\nOIS_dict'

In [5]:
# remove repetitive columns (incident type, officer id and subject id are in ID respectively)

force = force.drop(columns=["Occured_date_time", "ID"])


In [13]:
#force.Incident_Num.value_counts()

In [9]:
# create engine
cnx = sqlite3.connect('calls.sqlite')

In [10]:
# create cursor
cur = cnx.cursor()


In [12]:
# iterate through the dataframe 'force', make a bunch of new rows, and then make a dataframe out of those

new_row = []
#val_lst = []
for ix, row in force.iterrows():
    # query the 911 calls database
    # for each call, append to a bunch of records that we're building up into a dataframe
    
        
    value = row.epoch
    rowbeat = row.Beat
    
    if rowbeat == '-':
        vals = cur.execute(f"select * from log where epoch_calls BETWEEN ({value} - 120) AND {value} ").fetchall()
    else:
        vals = cur.execute(f"select * from log where epoch_calls BETWEEN ({value} - 3000) AND {value} ").fetchall()
    #val_lst.extend([value, val[0]])
    cnx.commit()
    
    for val in vals:
        new_row.extend( [[ix,  val[0], row.epoch, row.Incident_Num, row.Officer_ID, row.Subject_ID, val[1], val[2],
                        row.Incident_Type, row.Precinct, 
                        row.Sector, row.Beat, val[8], val[4],
                        row.Subject_Race, row.Subject_Gender, val[3], val[5], val[6], 
                        val[7], row.year, row.month, row.day] ])
  

In [13]:
# make dataframe from force dateframe and returned SQL table info (from calls table)

merged = pd.DataFrame(new_row, columns=["index","call_epoch", "force_epoch", "Incident_Num", "Officer_ID",
                                        "Subject_ID", "CAD_Event_Number", "Event_Clearance_Description",
                                "Force_Level", "Precinct", 
                                "Sector", "force_beat", "calls_beat", "Priority",
                                "Subject_Race", "Subject_Gender","Call_Type", "Initial_Call_Type", "Final_Call_Type",
                                "Original_Time_Queued", "year", "month","day"])



In [14]:
# if the rows have an associated beat and they don't match - drop them as a mismatched double
new_merged = []
for ix, row in merged.iterrows():
    forc = row.force_beat
    cal = row.calls_beat
    if forc != '-':
        if forc != cal:
            pass
        else:
            new_merged.append(row)
    else:
        new_merged.append(row)

In [15]:
# make it into a dataframe
new_merged = pd.DataFrame(new_merged)


In [15]:
'''# looking at duplicated CAD_E_Nums
dfrs = new_merged["CAD_Event_Number"].value_counts()
two = []
for v, i in zip(dfrs.values, dfrs.index):
    if v >1:
        two.append(i)
# len(two) == 362  
s = [x > 1 for x in dfrs]
dd =dfrs[s]
t = new_merged[new_merged["CAD_Event_Number"].isin(two)]'''


In [41]:
# make numeric
rev_df = new_merged.replace("-1", -1)
rev_df["Original_Time_Queued"] = pd.to_datetime(rev_df["Original_Time_Queued"])
rev_df["day_of_week"] = rev_df["Original_Time_Queued"].dt.dayofweek


In [42]:
# get hour
rev_df["hour"] = rev_df["Original_Time_Queued"].dt.hour

In [45]:
rev_df.Initial_Call_Type.head()

0            SUSPICIOUS PERSON, VEHICLE OR INCIDENT
1                        TRAFFIC - BLOCKING ROADWAY
2    MVC - HIT AND RUN (NON INJURY), INCLUDES IP/JO
3                        MVC - NON INJURY, BLOCKING
4          PARKING VIOLATION (EXCEPT ABANDONED CAR)
Name: Initial_Call_Type, dtype: object

In [57]:
# bring in rest of calls dataframe
calls = pd.read_csv("data/Call_Data.csv")

In [58]:
# establish columns related to time 
def callstime(df, col_arrive):
    yearmonthday = df[col_arrive].astype(str).str.split(" ")
    df["year"] = yearmonthday.apply(lambda x: x[2])
    df["day"] = yearmonthday.apply(lambda x: x[1])
    month_dict = {"Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4, "May": 5, "Jun": 6, "Jul": 7, "Aug": 8, "Sep": 9, "Oct": 10, "Nov": 11, "Dec":12}
    df["month"] = yearmonthday.apply(lambda x: month_dict[x[0]])
    time = yearmonthday.apply(lambda x: x[3])
    #tim = time.astype(str).str.split(":")
    #df["hour"] = tim.apply(lambda x: x[0])
    return df

calls = callstime(calls,"Arrived Time")



In [79]:
# make the df smaller 
calls17 = pd.DataFrame(calls[calls["year"].isin(["2014", "2015", "2016", "2017", "2018", "2019"])])


In [20]:
#s = pd.to_datetime(calls17["Original Time Queued"], infer_datetime_format=True)
#(calls17["time"])

#.apply(lambda x: pd.datetime.strftime(x, '%H:%M:%S'))

#s = s.apply(lambda x: pd.datetime.strftime(x, '%H'))

In [81]:
calls17.columns
calls17.drop("hour", axis=1, inplace=True)

In [82]:
# change O_t_q to dt and then day of week
calls17["Original_Time_Queued"] = pd.to_datetime(calls17["Original Time Queued"])
calls17["day_of_week"] = calls17["Original_Time_Queued"].dt.dayofweek

# get hour
calls17["hour"] = calls17["Original_Time_Queued"].dt.hour

In [83]:
calls17.drop("Original Time Queued", axis=1, inplace=True)

Index(['CAD Event Number', 'Event Clearance Description', 'Call Type',
       'Priority', 'Initial Call Type', 'Final Call Type', 'Arrived Time',
       'Precinct', 'Sector', 'Beat', 'year', 'day', 'month',
       'Original_Time_Queued', 'day_of_week', 'hour'],
      dtype='object')

In [85]:
# send calls17 df to pickle so I don't have to keep doing this:
# calls17.to_pickle("data/calls17_cleaned.pkl")

In [ ]:
# import from pickle!!

# calls_17 = pd.read_pickle("data/calls17_cleaned.pkl")

In [88]:
call_cols = calls17.columns

In [87]:
# add in rows from Calls17 sql table that are not already represented in merged df
add_call_rows = []

series = list(rev_df["CAD_Event_Number"])
#val_lst = []
for ix, row in calls17.iterrows():
    # query the 911 calls database
    # for each call, append to a bunch of records that we're building up into a dataframe
    
        
    CAD = row["CAD Event Number"]
    #rowbeat = row.Beat
    series 
    
    if CAD in series:
        pass
    else:
        add_call_rows.extend([row])
    #val_lst.extend([value, val[0]])

'''    
    for val in vals:
        new_row.extend( [[ix,  val[0], row.epoch, row.Incident_Num, row.Officer_ID, row.Subject_ID, val[1], val[2],
                        row.Incident_Type, row.Precinct, 
                        row.Sector, row.Beat, val[8], val[4],
                        row.Subject_Race, row.Subject_Gender, val[3], val[5], val[6], 
                        val[7], row.year, row.month, row.day, row.time] ])'''



'    \n    for val in vals:\n        new_row.extend( [[ix,  val[0], row.epoch, row.Incident_Num, row.Officer_ID, row.Subject_ID, val[1], val[2],\n                        row.Incident_Type, row.Precinct, \n                        row.Sector, row.Beat, val[8], val[4],\n                        row.Subject_Race, row.Subject_Gender, val[3], val[5], val[6], \n                        val[7], row.year, row.month, row.day, row.time] ])'

In [101]:
calls_add = pd.DataFrame(add_call_rows, columns = call_cols)


In [93]:
# pickle this df where it is just the rows not in rev_df so I can do more eda later:
# calls17.to_pickle("data/calls17_cleaned_not_in_rev_df.pkl")

In [28]:
# change O_t_q to dt
# s = pd.to_datetime(calls_add["Original_Time_Queued"],infer_datetime_format=True)


In [29]:
# calls_add["day_of_week"] = s.dt.dayofweek


In [102]:
# if I find percentages and then disaggregate - can have not null genders and races here and in the rest

calls_add["Force_Level"] = 0
calls_add["Subject_Gender"] = "-"
calls_add["Subject_Race"] = "-"


In [103]:


# need to change column names and add columns to match rev_df
calls_add.rename(index=str, columns={"CAD Event Number": "CAD_Event_Number", "Event Clearance Description": "Event_Clearance_Description",
                                    "Call Type": "Call_Type", "Initial Call Type": "Initial_Call_Type"}, inplace=True)

calls_add.drop(["Priority", "Final Call Type", "Arrived Time", 'year'], axis=1, inplace=True )


In [47]:
# reorder the smaller df obviously




In [78]:
# drop columns I don't need for this model:


rev_df.drop(["index", "call_epoch", "force_epoch", "Incident_Num", "Officer_ID", "Subject_ID", 
          #  "Original_Time_Queued"], axis=1, inplace=True)

Index(['CAD Event Number', 'Event Clearance Description', 'Call Type',
       'Initial Call Type', 'Precinct', 'Sector', 'Beat', 'day', 'month',
       'Original_Time_Queued', 'day_of_week', 'hour'],
      dtype='object')

In [49]:
rev_df.drop(["Priority", "Final_Call_Type", "year"], axis=1, inplace=True)

In [50]:

rev_df.drop("force_beat", axis=1, inplace=True)

In [36]:
rev_df.rename(index=str, columns={"calls_beat":"Beat"}, inplace=True)
rev_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8616 entries, 0 to 262504
Data columns (total 13 columns):
CAD_Event_Number               8616 non-null int64
Event_Clearance_Description    8616 non-null object
Force_Level                    8616 non-null object
Precinct                       8616 non-null object
Sector                         8616 non-null object
Beat                           8616 non-null object
Subject_Race                   8616 non-null object
Subject_Gender                 8616 non-null object
Call_Type                      8616 non-null object
Initial_Call_Type              8616 non-null object
month                          8616 non-null object
day                            8616 non-null object
day_of_week                    8616 non-null int64
dtypes: int64(2), object(11)
memory usage: 942.4+ KB


In [105]:
calls_add.drop(["Original_Time_Queued"], axis=1, inplace=True)


<class 'pandas.core.frame.DataFrame'>
Index: 1451138 entries, 1674201 to 3986667
Data columns (total 14 columns):
CAD_Event_Number               1451138 non-null int64
Event_Clearance_Description    1451138 non-null object
Call_Type                      1451138 non-null object
Initial_Call_Type              1451138 non-null object
Precinct                       1451138 non-null object
Sector                         1441257 non-null object
Beat                           1451138 non-null object
day                            1451138 non-null object
month                          1451138 non-null int64
day_of_week                    1451138 non-null int64
hour                           1451138 non-null int64
Force_Level                    1451138 non-null int64
Subject_Gender                 1451138 non-null object
Subject_Race                   1451138 non-null object
dtypes: int64(5), object(9)
memory usage: 166.1+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8616 entries, 0 to 

(None, None)

In [106]:
#reorder columns so they match
calls_add = calls_add[["CAD_Event_Number", "Event_Clearance_Description", "Force_Level", "Precinct", "Sector", "Beat",
                      "Subject_Race", "Subject_Gender","Call_Type", "Initial_Call_Type", "month", "day", "day_of_week", "hour"]]
calls_add.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1451138 entries, 1674201 to 3986667
Data columns (total 14 columns):
CAD_Event_Number               1451138 non-null int64
Event_Clearance_Description    1451138 non-null object
Force_Level                    1451138 non-null int64
Precinct                       1451138 non-null object
Sector                         1441257 non-null object
Beat                           1451138 non-null object
Subject_Race                   1451138 non-null object
Subject_Gender                 1451138 non-null object
Call_Type                      1451138 non-null object
Initial_Call_Type              1451138 non-null object
month                          1451138 non-null int64
day                            1451138 non-null object
day_of_week                    1451138 non-null int64
hour                           1451138 non-null int64
dtypes: int64(5), object(9)
memory usage: 166.1+ MB


In [107]:
calls_add.info(), rev_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1451138 entries, 1674201 to 3986667
Data columns (total 14 columns):
CAD_Event_Number               1451138 non-null int64
Event_Clearance_Description    1451138 non-null object
Force_Level                    1451138 non-null int64
Precinct                       1451138 non-null object
Sector                         1441257 non-null object
Beat                           1451138 non-null object
Subject_Race                   1451138 non-null object
Subject_Gender                 1451138 non-null object
Call_Type                      1451138 non-null object
Initial_Call_Type              1451138 non-null object
month                          1451138 non-null int64
day                            1451138 non-null object
day_of_week                    1451138 non-null int64
hour                           1451138 non-null int64
dtypes: int64(5), object(9)
memory usage: 166.1+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8616 entries, 0 to 

(None, None)

In [108]:

nm = pd.concat([rev_df, calls_add])

In [109]:
# pickle this so I don't have to do this again:

nm.to_pickle("data/all_calls17_uof_df.pkl")

In [110]:
nm.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1459754 entries, 0 to 3986667
Data columns (total 14 columns):
CAD_Event_Number               1459754 non-null int64
Event_Clearance_Description    1459754 non-null object
Force_Level                    1459754 non-null object
Precinct                       1459754 non-null object
Sector                         1449873 non-null object
Beat                           1459754 non-null object
Subject_Race                   1459754 non-null object
Subject_Gender                 1459754 non-null object
Call_Type                      1459754 non-null object
Initial_Call_Type              1459754 non-null object
month                          1459754 non-null object
day                            1459754 non-null object
day_of_week                    1459754 non-null int64
hour                           1459754 non-null int64
dtypes: int64(3), object(11)
memory usage: 167.1+ MB


In [86]:
# not sure I want to do this

init = nm["Initial_Call_Type"]
to_drop = init[init.str.startswith("-ASSIGNED DUTY")]


s = nm["Initial_Call_Type"].str.replace(" - .*", "", regex=True)
nm["Initial_Call_Type"] = s

In [111]:
# correct time columns 

#rev_df["year"] = rev_df["year"].astype(int)
nm["month"] = nm["month"].astype(int)
nm["day"] = nm["day"].astype(int)
#nm["day_of_week"] = nm.Original_Time_Queued.dt.dayofweek

# nm['call_hour'] = nm.Original_Time_Queued.apply(lambda x: x.hour)
#rev_df['arrive_hour'] = rev_df.time.apply(lambda x: x.hour)
#rev_df['time_to_arrival'] = rev_df["arrive_hour"] - rev_df["call_hour"]
#rev_df.drop("arrive_hour", axis=1, inplace=True)

In [112]:
# make second df and keep leaky data for exploration later
# take out event description and final call (bc you wouldn't know)
nm_saved = nm.copy(deep=True)


In [116]:
# need to remove ECD for now - leaky!

nm.drop("Event_Clearance_Description", axis=1, inplace=True)



KeyError: "['Event_Clearance_Description'] not found in axis"

In [119]:
nm.Initial_Call_Type.value_counts()

PREMISE CHECK, OFFICER INITIATED ONVIEW ONLY        135271
SUSPICIOUS PERSON, VEHICLE OR INCIDENT               85572
TRAFFIC STOP - OFFICER INITIATED ONVIEW              81414
DISTURBANCE, MISCELLANEOUS/OTHER                     71994
OFF DUTY EMPLOYMENT                                  55520
THEFT (DOES NOT INCLUDE SHOPLIFT OR SVCS)            43115
DIRECTED PATROL ACTIVITY                             40385
NUISANCE - MISCHIEF                                  36101
SUSPICIOUS STOP - OFFICER INITIATED ONVIEW           35141
PARKING VIOLATION (EXCEPT ABANDONED CAR)             33563
TRESPASS                                             32540
FOLLOW UP                                            27953
SHOPLIFT - THEFT                                     23039
BURG - RES (INCL UNOCC STRUCTURES ON PROP)           22616
ALARM - RESIDENTIAL - BURGLARY, SILENT/AUDIBLE       21619
ALARM - COMM, SILENT/AUD BURG (INCL BANKS)           20917
NARCOTICS - VIOLATIONS (LOITER, USE, SELL, NARS)     203

In [144]:
# cool EDA

call_types = nm.groupby(["Call_Type"]).Initial_Call_Type.value_counts()

df1 = nm[nm['Initial_Call_Type'].str.contains("FRAUD") ]


call_types["911"]
df1

,CAD_Event_Number,Force_Level,Precinct,Sector,Beat,Subject_Race,Subject_Gender,Call_Type,Initial_Call_Type,month,day,day_of_week,hour
27,2017000296473,Level 1 - Use of Force,-,-,N3,White,Male,ONVIEW,"FRAUD - FORGERY,BUNCO, SCAMS, ID THEFT, ETC",8,11,4,12
127,2019000097105,Level 1 - Use of Force,-,-,D1,Not Specified,Male,911,"FRAUD - FORGERY,BUNCO, SCAMS, ID THEFT, ETC",3,18,0,16
486,2017000090733,Level 1 - Use of Force,E,C,C3,Not Specified,Female,"TELEPHONE OTHER, NOT 911","FRAUD - FORGERY,BUNCO, SCAMS, ID THEFT, ETC",3,14,1,13
17631,2014000157843,Level 2 - Use of Force,N,U,U1,White,Male,"TELEPHONE OTHER, NOT 911","FRAUD - FORGERY,BUNCO, SCAMS, ID THEFT, ETC",5,20,1,15
17681,2014000157843,Level 2 - Use of Force,N,U,U1,White,Male,"TELEPHONE OTHER, NOT 911","FRAUD - FORGERY,BUNCO, SCAMS, ID THEFT, ETC",5,20,1,15
25082,2018000228040,Level 1 - Use of Force,N,N,N2,White,Female,ONVIEW,"FRAUD - FORGERY,BUNCO, SCAMS, ID THEFT, ETC",6,22,4,14
42323,2018000388501,Level 1 - Use of Force,S,S,S2,Black or African American,Male,"TELEPHONE OTHER, NOT 911","FRAUD - FORGERY,BUNCO, SCAMS, ID THEFT, ETC",10,17,2,8
44619,2014000387668,Level 2 - Use of Force,W,D,D2,White,Male,ONVIEW,"FRAUD - FORGERY,BUNCO, SCAMS, ID THEFT, ETC",11,19,2,18
50230,2014000417879,Level 2 - Use of Force,N,N,N1,White,Male,ONVIEW,"FRAUD - FORGERY,BUNCO, SCAMS, ID THEFT, ETC",12,18,3,12
65124,2014000288740,Level 2 - Use of Force,S,S,S1,White,Female,"TELEPHONE OTHER, NOT 911","FRAUD - FORGERY,BUNCO, SCAMS, ID THEFT, ETC",8,29,4,17


In [139]:
def initial_call_type_cats(value):    
    if "MVC" in value:
        return "MVC"
    elif 
    

In [140]:
df = pd.DataFrame({"col": ["a", "MVC-a"]})
new_col = []
for idx, row in df.iterrows():
    value = row.col
    new_col.append(initial_call_type_cats(value))
    

In [141]:
new_col

[None, 'MVC']

In [56]:
# just for this model - need to alter in the future to keep more data with which to play - especially "call type in calls17"

calls_add.info(), rev_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1451138 entries, 1674201 to 3986667
Data columns (total 12 columns):
CAD_Event_Number               1451138 non-null int64
Event_Clearance_Description    1451138 non-null object
Force_Level                    1451138 non-null int64
Precinct                       1451138 non-null object
Sector                         1441257 non-null object
Beat                           1451138 non-null object
Subject_Race                   1451138 non-null object
Subject_Gender                 1451138 non-null object
Initial_Call_Type              1451138 non-null object
Original_Time_Queued           1451138 non-null datetime64[ns]
month                          1451138 non-null int64
day                            1451138 non-null object
dtypes: datetime64[ns](1), int64(3), object(8)
memory usage: 143.9+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8616 entries, 0 to 262504
Data columns (total 14 columns):
CAD_Event_Number               8616 non-n

(None, None)

In [796]:
# make dummy columns for categorical 
nm = pd.get_dummies(data = nm, 
                     columns=["Precinct", "Sector", "Beat", "Subject_Gender", "Subject_Race",
                               "Initial_Call_Type"], 
                     prefix=["P",  "S", "B", "SG", "SR","IC"])
                              



# pd.get_dummies(data=df, columns=['A', 'B'])
                      

In [798]:
'''# drop leaky data
rev_df.drop(["Event_Clearance_Description","Final_Call_Type", "time ", "index", "Incident_Num", "Officer_ID",
            "Subject_ID", "CAD_Event_Number", "Force_Level", "Priority", "force_epoch", "calls_beat", "Call_Type", 
             "Original_Time_Queued", "time", "year", "call_epoch","forceb_-","Gender_Not Specified", 
             "Race_Not Specified", "Precinct", "Sector"], axis=1, inplace=True )

'''

In [799]:
#rev_df.drop(indicest, axis=0, inplace=True)

len(rev_df_leaky), len(rev_df), len(new_merged)

(8600, 8600, 8600)

In [823]:
rev_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8600 entries, 0 to 262504
Data columns (total 177 columns):
month                                                      int64
day                                                        int64
forceb_99                                                  uint8
forceb_B1                                                  uint8
forceb_B2                                                  uint8
forceb_B3                                                  uint8
forceb_C1                                                  uint8
forceb_C2                                                  uint8
forceb_C3                                                  uint8
forceb_D1                                                  uint8
forceb_D2                                                  uint8
forceb_D3                                                  uint8
forceb_E1                                                  uint8
forceb_E2                                    

In [800]:
clf = RandomForestClassifier(n_estimators=100, random_state=659, oob_score=True)



In [801]:
# X for classifier
X = rev_df

#X.info(verbose=True)
X.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8600 entries, 0 to 262504
Data columns (total 177 columns):
month                                                      int64
day                                                        int64
forceb_99                                                  uint8
forceb_B1                                                  uint8
forceb_B2                                                  uint8
forceb_B3                                                  uint8
forceb_C1                                                  uint8
forceb_C2                                                  uint8
forceb_C3                                                  uint8
forceb_D1                                                  uint8
forceb_D2                                                  uint8
forceb_D3                                                  uint8
forceb_E1                                                  uint8
forceb_E2                                    

In [802]:
cols = X.columns

In [803]:
X_scaled = StandardScaler().fit_transform(X)

In [804]:
y = new_merged.Force_Level
y.replace({"Level 1 - Use of Force":1,"Level 2 - Use of Force":2,"Level 3 - OIS":4, "Level 3 - Use of Force": 3 })
y.shape, X_scaled.shape

((8600,), (8600, 177))

In [805]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=659)


In [806]:
clf.fit(X_train, y_train) #takes three minutes

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=True, random_state=659, verbose=0,
                       warm_start=False)

In [807]:
len(clf.feature_importances_), len(cols)

(177, 177)

In [808]:
# most important features
indices = np.argsort(clf.feature_importances_)
big = indices[-1:-20:-1]

cols[big]

Index(['day', 'call_hour', 'month', 'day_of_week',
       'Race_Black or African American', 'Race_White', 'forceb_E2',
       'forceb_M3', 'Gender_Female', 'forceb_K2', 'Gender_Male', 'forceb_M1',
       'InitCall_DISTURBANCE, MISCELLANEOUS/OTHER', 'InitCall_ASLT',
       'InitCall_PREMISE CHECK, OFFICER INITIATED ONVIEW ONLY', 'forceb_K3',
       'forceb_M2', 'InitCall_SUSPICIOUS PERSON, VEHICLE OR INCIDENT',
       'forceb_D1'],
      dtype='object')

In [809]:
# least important features
little = indices[:10]
cols[little]


Index(['InitCall_TEST CALL ONLY', 'InitCall_ANIMAL, REPORT',
       'InitCall_EXPLOSION', 'InitCall_INFORMATIONAL BROADCASTS',
       'InitCall_POWER OUT', 'InitCall_BOMB THREATS',
       'InitCall_SHOTS -DELAY/INCLUDES HEARD/NO ASSAULT',
       'InitCall_SLEEPER ABOARD BUS/COMMUTER TRAIN', 'InitCall_RAPE',
       'InitCall_WARRANT PICKUP'],
      dtype='object')

In [810]:
clf.oob_score_ # accuracy of training set

0.8989825581395349

In [811]:
yhat = clf.predict(X_test)

In [812]:
# if I were going to use this to predict the outcome, it would have to take human input and then run it through 
# all the things and make it a very long array



In [813]:
sum(yhat == y_test) / len(yhat) # accuracy of testing set

0.8930232558139535

In [814]:
pp = clf.predict_proba(X_test)

In [815]:
clf.classes_

array(['Level 1 - Use of Force', 'Level 2 - Use of Force',
       'Level 3 - OIS', 'Level 3 - Use of Force'], dtype=object)

In [816]:
for p in pp[:10]:
    print(f"Pr of Force 1 is: {p[0]}, Pr of Force 2: {p[1]}, Pr of Force 3: {p[3]}, & Pr of Officer Involved Shooting: {p[2]}")
          
          

Pr of Force 1 is: 0.7835, Pr of Force 2: 0.19649999999999998, Pr of Force 3: 0.02, & Pr of Officer Involved Shooting: 0.0
Pr of Force 1 is: 0.09, Pr of Force 2: 0.04, Pr of Force 3: 0.87, & Pr of Officer Involved Shooting: 0.0
Pr of Force 1 is: 0.282, Pr of Force 2: 0.718, Pr of Force 3: 0.0, & Pr of Officer Involved Shooting: 0.0
Pr of Force 1 is: 0.09, Pr of Force 2: 0.02, Pr of Force 3: 0.89, & Pr of Officer Involved Shooting: 0.0
Pr of Force 1 is: 0.845, Pr of Force 2: 0.125, Pr of Force 3: 0.03, & Pr of Officer Involved Shooting: 0.0
Pr of Force 1 is: 0.99, Pr of Force 2: 0.01, Pr of Force 3: 0.0, & Pr of Officer Involved Shooting: 0.0
Pr of Force 1 is: 1.0, Pr of Force 2: 0.0, Pr of Force 3: 0.0, & Pr of Officer Involved Shooting: 0.0
Pr of Force 1 is: 1.0, Pr of Force 2: 0.0, Pr of Force 3: 0.0, & Pr of Officer Involved Shooting: 0.0
Pr of Force 1 is: 0.935, Pr of Force 2: 0.065, Pr of Force 3: 0.0, & Pr of Officer Involved Shooting: 0.0
Pr of Force 1 is: 0.97, Pr of Force 2: 0.

In [817]:
rdl = rev_df_leaky.copy(deep=True)

In [818]:
race = rdl.groupby("Subject_Race")

In [819]:
rdl["Force_Level"].value_counts()

Level 1 - Use of Force    6303
Level 2 - Use of Force    2108
Level 3 - Use of Force     125
Level 3 - OIS               64
Name: Force_Level, dtype: int64

In [820]:
force.columns

Index(['Incident_Num', 'Incident_Type', 'Precinct', 'Sector', 'Beat',
       'Officer_ID', 'Subject_ID', 'Subject_Race', 'Subject_Gender', 'year',
       'month', 'day', 'time', 'epoch'],
      dtype='object')

In [524]:
force["Incident_Type"].value_counts()

Level 1 - Use of Force    7474
Level 2 - Use of Force    2185
Level 3 - Use of Force     105
Level 3 - OIS               74
Name: Incident_Type, dtype: int64

In [525]:
# need to do gradient boosting to do partial dependency plotting
gdbr = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
                                  max_depth=1, random_state=0).fit(X_scaled, y)


In [526]:
#gdbr.feature_importances_

indices_gdbr = np.argsort(gdbr.feature_importances_)
big_gdbr = indices[-1:-10:-1]

cols[big_gdbr]


Index(['day', 'call_hour', 'month', 'day_of_week', 'Race_White',
       'Race_Black or African American', 'CallType_911', 'CallType_ONVIEW',
       'Race_Not Specified'],
      dtype='object')

In [ ]:
force3[force3.month == 12]

In [ ]:
plt.hist(force3.month, bins=12);

In [ ]:
list(big_gdbr)

In [ ]:

# make plots
my_plots = plot_partial_dependence(gdbr,       
                                   features=[131, 5361], # column numbers of plots to show
                                   X=X,            # raw predictors data.
                                   #feature_names=['day', 'month', 'day_of_week'],
                                   target=["Level 1 - Use of Force"],# for multi-class must use label
                                   grid_resolution=10)

In [ ]:
#gdbr.predict(X_test) 
#gdbr.predict_proba(X_test)
gdbr.score(X_test, y_test)

In [ ]:

importance = np.std([tree.feature_importances_
                            for tree in clf.estimators_], axis=0)

r = range(importance.shape[0])
f = clf.feature_importances_
yerr = importance

# Extend the code to find the standard deviation of the importance for each feature across all trees. 
fig, ax = plt.subplots(1,1, figsize=(20,8), sharex=True)
#axs.flatten()

ax.errorbar(r,f, yerr=yerr)
ax.set_title('all errorbars')


#fig.suptitle('Errors')

plt.tight_layout()
plt.xticks(np.arange(len(r)), X.columns, rotation="vertical")
plt.show();
#plt.bar(r, f, yerr=yerr, align="center");




In [ ]:
'''force3.Incident_Num.value_counts()
t = force3[force3["Incident_Num"] == 12308]
#t.drop_duplicates("CAD_Event_Number", inplace=True) # 'force_beat', 'calls_beat'
len(t.CAD_Event_Number.value_counts()), len(t)
t2 = t[t.force_beat.values == t.calls_beat.values]
t2'''

'''force3 = merged[merged.Force_Level == "Level 3 - Use of Force"]
force2 = merged[merged.Force_Level == "Level 2 - Use of Force"]
force1 = merged[merged.Force_Level == "Level 1 - Use of Force"]
len(force3), len(force2), len(force1), len(merged)'''

'''# group by incident number first

x = force3.Subject_Race.value_counts()
lst = list(x.values)
plt.bar(np.arange(7), lst)
#plt.xticks([])
#plt.xticks()

plt.xticks((0,1, 2, 3,4,5,6), ('White', 'Black or African American ', 'Not Specified ','Hispanic or Latino',
                               'Asian','American Indian/Alaska Native ','Nat Hawaiian/Oth Pac Islander'), 
          rotation='vertical')
plt.title("Level 3 Force by Race 2014-2019");'''

In [ ]:
'''force3beats = force3[force3.force_beat.values == force3.calls_beat.values]
len(force3beats)
f3b = force3beats.drop(["index", "Force_Level", "Precinct"], axis=1)
f3b.drop("Sector", axis=1, inplace=True)
#f3b '''

In [ ]:
'''#rev_df.drop(["Subject_Gender", "Subject_Race"], axis=1, inplace=True)
rev_df.info(verbose=True)'''

In [ ]:
'''y = force3.Subject_Gender.value_counts()
lst = list(y.values)
plt.bar(np.arange(2), lst)
#plt.xticks([])
#plt.xticks()

plt.xticks((0,1), ('Male', 'Female'), 
          rotation='horizontal')
plt.title("Level 3 Force by Gender 2014-2019");'''

In [822]:
new_merged["Initial_Call_Type"].value_counts()

PREMISE CHECK, OFFICER INITIATED ONVIEW ONLY      718
DISTURBANCE, MISCELLANEOUS/OTHER                  696
ASLT                                              520
SUSPICIOUS PERSON, VEHICLE OR INCIDENT            439
TRAFFIC STOP                                      439
OFF DUTY EMPLOYMENT                               282
FIGHT                                             278
SUSPICIOUS STOP                                   260
MVC                                               256
DIST                                              228
THEFT (DOES NOT INCLUDE SHOPLIFT OR SVCS)         211
TRESPASS                                          204
BURG                                              185
AUTO RECOVERY                                     179
DIRECTED PATROL ACTIVITY                          161
PROPERTY                                          154
ALARM                                             147
WARRANT                                           146
FOLLOW UP                   

Make a way to enter in info


In [705]:
# time info[day, day of the week, call hour, month], location[beat, sector, precinct], gender, race, 
# initial call --> drop down menu
# could change later to just ask for address

def new_call(gender, race, reason, beat):
    now = datetime.datetime.now()
    month = now.month
    day = now.day
    hour = now.hour
    day_of_week = now.dt.dayofweek
    


2019 7 6 18


What is your street address?1201 summit ave
